In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from json import JSONDecodeError

In [2]:
project_filters = {
    "location": "",
    "coordinates": [
        [
            "",
            ""
        ]
    ],
    "range": "2",
    "priceRangeLower": "0",
    "priceRangeUpper": "0",
    "flatType": "",
    "waitingTime": "",
    "modeOfSale": "SBF",
    "floorRange": "",
    "remainingLeaseRangeLower": 1,
    "remainingLeaseRangeUpper": 99,
    "ethnicGroup": "",
    "rank": "Location, Price Range, Flat Type, Remaining Lease",
    "town": ""
}

In [15]:
def read_header(header_name):
    with open(header_name) as f:
        header_get_projects = {}
        for line in f:
            key_value = line.replace("\n", "").split(":")
            header_get_projects[key_value[0]] = key_value[1]
    return header_get_projects


def get_all_projects_data(header_get_projects, project_filters):
    r = requests.post(
        "https://homes.hdb.gov.sg/home-api/public/v1/map/getCoordinatesByFilters", 
        headers=header_get_projects, json=project_filters)
    try:
        data = r.json()
    except JSONDecodeError:
        print(r.content)
        raise JSONDecodeError
    return data

def get_ethnic_quotas(header_get_projects, project_id):
    r = requests.post(
        "https://homes.hdb.gov.sg/home-api/public/v1/launch/getProjectAvailabilityAndEthnic", 
        headers=header_get_projects, json=project_id)
    try:
        data = r.json()
    except JSONDecodeError:
        print(r.content)
        raise JSONDecodeError
    else:
        data["project_id"] = project_id
    return data


def get_required_project_data(project_data):
    required_data = {} 
    required_data["coordinates"] = project_data["coordinates"]
    properties = project_data["properties"]
    required_data["project"] = properties["address"]
    
    required_data["region"] = properties["region"]
    description = properties["description"][0]
    try:
        description1 = properties["description"][1]
    except IndexError:
        pass
    else:
        print(description1)
    required_data["town"] = description["town"]
    required_data["project_id"] = description["projectId"]        
    return required_data

def get_all_required_projects_data(all_projects_data):
    all_required_projects_data = []
    for project_data in all_projects_data:
        required_project_data = get_required_project_data(project_data)
        all_required_projects_data.append(required_project_data)

    return all_required_projects_data

def get_all_units_xml_in_project(project_id, header_get_projects):
    r = requests.post(
        "https://homes.hdb.gov.sg/home-api/public/v1/launch/getUnitXml", 
        headers=header_get_projects, 
        json={"projectId":project_id})
    unit_xml = ET.fromstring(r.content)
    return unit_xml

def get_project_units(unit_xml):
    units = unit_xml.findall("./standard-project-info/unit-details/unit")
    return units
def get_project_name(unit_xml):
    project_name = unit_xml.find("./project-name").text
    return project_name
def get_project_streetname(unit_xml):
    street_name = unit_xml.find("./standard-project-info/key/street-name").text
    return street_name
def get_remaining_lease(unit_xml):
    remaining_lease = unit_xml.find("./standard-project-info/key/remaining-lease").text
    return remaining_lease
def get_unit_details(unit):
    unit_details = {}
    for unit_detail in unit:
        unit_details[unit_detail.tag] = unit_detail.text
    return unit_details

def get_all_unit_details_in_project(unit_xml, required_project_data):
    project_units = get_project_units(unit_xml)
    street_name = get_project_streetname(unit_xml)
    remaining_lease = get_remaining_lease(unit_xml)
    project_name = get_project_name(unit_xml)
    unit_details_1 = {
        "remaining_lease": remaining_lease, 
        "street_name": street_name,
        "project_name": project_name}
    all_unit_details_for_project = []
    for unit in project_units:
        unit_details_2 = get_unit_details(unit)
        unit_details = {**required_project_data, **unit_details_1, **unit_details_2}
        all_unit_details_for_project.append(unit_details)
    return all_unit_details_for_project

def get_all_unit_details_for_all_projects(all_required_projects_data):
    all_unit_details_for_all_projects = []
    projects_ethnic_quota_data = []
    for i, required_project_data in enumerate(all_required_projects_data):
        print(f"""{i+1} {required_project_data["project"]}""")
        project_id = required_project_data["project_id"]
        all_units_xml = get_all_units_xml_in_project(project_id, header_get_projects)
        all_unit_details_in_project = get_all_unit_details_in_project(all_units_xml, required_project_data)

        project_ethnic_quota_data = get_ethnic_quotas(header_get_projects, project_id)

        all_unit_details_for_all_projects.extend(all_unit_details_in_project)
        projects_ethnic_quota_data.append(project_ethnic_quota_data)
    return all_unit_details_for_all_projects, projects_ethnic_quota_data

In [16]:
header_get_projects = read_header("header_get_projects")
header_get_projects

{'authority': 'homes.hdb.gov.sg',
 'accept': 'application/json, text/plain, */*',
 'content-type': 'application/json',
 'origin': 'https',
 'referer': 'https',
 'sessionid': 'LO165365523118802t874vxAA9XjL2f',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53'}

In [18]:
all_projects_data = get_all_projects_data(header_get_projects, project_filters)
all_required_projects_data = get_all_required_projects_data(all_projects_data)
result = get_all_unit_details_for_all_projects(all_required_projects_data)


1 Sunshine Court
2 Sunshine Gardens
3 Limbang Green
4 Choa Chu Kang Dr
5 Keat Hong Verge
6 Keat Hong Axis
7 Teck Whye Vista
8 Teck Whye View
9 Teck Whye Heights II
10 Choa Chu Kang Ave 3
11 Clementi West St 2
12 West Coast Rd
13 Clementi West St 2
14 Sunset Way Residence
15 Clementi Northarc
16 Clementi Gateway
17 Clementi Ave 4
18 Clementi Shine
19 Clementi Ave 5
20 Clementi Ave 5
21 Clementi Ave 5
22 Trivelis
23 Clementi Crest
24 Clementi Peaks
25 Clementi Heights
26 West Coast Vista
27 Clementi West St 1
28 Waterloo Centre
29 Cantonment Towers
30 Blangah Garden
31 Blangah Court
32 Telok Blangah Beacon
33 Telok Blangah Towers
34 Telok Blangah Parcview
35 Lengkok Bahru
36 Bukit Purmei Ville
37 Jln Bt Merah
38 Kim Tian Towers
39 Havelock View
40 Boon Tiong Arcadia
41 Golden Tulip
42 City Vue @henderson
43 Bishan St 13
44 Golden Jasmine
45 Bishan Ridges
46 Bishan Towers
47 Bishan St 12
48 Bishan View
49 Bishan Green
50 Bishan St 24
51 Bt Batok West Ave 6
52 Bt Batok West Ave 8
53 Golden

In [19]:
flat_data = result[0]
ethnic_quota_data = result[1]

In [20]:
flat_data_df = pd.DataFrame.from_dict(flat_data)
flat_data_df.to_csv("unit_details.csv", index=False)

In [90]:
flat_data_df

,coordinates,project,region,town,project_id,remaining_lease,street_name,project_name,flat-type,block-number,unit-id,unit-number,level,area,deliveryPossessionDate,estimatedCompletionDate,price,short-lease-price,price-tooltip,re-purchase-tooltip
0,"[1.3766745369726854, 103.73789561376547]",Sunshine Court,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE0XzE2NTAzNDk4MDAwNTM,15 - 45 years,Choa Chu Kang Ave 5,Sunshine Court,2-Room Flexi,476A,2022-05_SBF_476A_02_01,01,02,36,Keys Available,-,57200,NaN,"<![CDATA[<br>$57,200 - 45 Years <br>$55,000 - ...",You may view floor plans and photographs (if a...
1,"[1.3766745369726854, 103.73789561376547]",Sunshine Court,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE0XzE2NTAzNDk4MDAwNTM,15 - 45 years,Choa Chu Kang Ave 5,Sunshine Court,2-Room Flexi,476A,2022-05_SBF_476A_09_07,07,09,47,Keys Available,-,84400,NaN,"<![CDATA[<br>$84,400 - 45 Years <br>$80,900 - ...",You may view floor plans and photographs (if a...
2,"[1.3765873889910192, 103.74007753792313]",Sunshine Gardens,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE1XzE2NTMzNzM4MDAwNDA,94 years,Choa Chu Kang Ave 5,Sunshine Gardens,5-Room,488C,2022-05_SBF_488C_02_161,161,02,113,Keys Available,-,405000,NaN,NaN,NaN
3,"[1.3765873889910192, 103.74007753792313]",Sunshine Gardens,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE1XzE2NTMzNzM4MDAwNDA,94 years,Choa Chu Kang Ave 5,Sunshine Gardens,5-Room,488C,2022-05_SBF_488C_02_163,163,02,113,Keys Available,-,405000,NaN,NaN,NaN
4,"[1.3942738802398278, 103.74452794822368]",Limbang Green,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX041QzEyXzE2NTAzNDYyMDAwODg,15 - 45 years,Choa Chu Kang St 52,Limbang Green,2-Room Flexi,570,2022-05_SBF_570_02_224,224,02,47,Keys Available,-,88500,NaN,"<![CDATA[<br>$88,500 - 45 Years <br>$84,800 - ...",You may view floor plans and photographs (if a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,"[1.418883880805233, 103.84487594211994]",Acacia Breeze @ Yishun,NORTH REGION,Yishun,2022-05_SBF_WVNfTjVDMl8xNjUwMzQ2MjAwMDgy,93 years,Yishun St 51,Acacia Breeze @ Yishun,"3-Room (Income Ceiling $7,000)",505A,2022-05_SBF_505A_07_10,10,07,68,Keys Available,-,253000,NaN,NaN,You may view floor plans and photographs (if a...
1948,"[1.413868249473606, 103.84013548258679]",Angsana Breeze @ Yishun,NORTH REGION,Yishun,2022-05_SBF_WVNfTjVDNF8xNjUwMzUzNDAwMDM1,95 years,Yishun Ave 4,Angsana Breeze @ Yishun,4-Room,509B,2022-05_SBF_509B_03_20,20,03,93,Keys Available,-,338000,NaN,NaN,You may view floor plans and photographs (if a...
1949,"[1.4156731558147704, 103.84143663027895]",Oleander Breeze @ Yishun,NORTH REGION,Yishun,2022-05_SBF_WVNfTjVDNl8xNjUwMzQyNjAwMDI3,95 years,Yishun St 51,Oleander Breeze @ Yishun,4-Room,511A,2022-05_SBF_511A_03_409,409,03,93,Keys Available,-,353000,NaN,NaN,You may view floor plans and photographs (if a...
1950,"[1.4193039750769822, 103.83535078902591]",Yishun Ring Rd,NORTH REGION,Yishun,2022-05_SBF_WVNfTjZDMl8xNjUwMzM5MDAwMTYy,65 years,Yishun Ring Rd,Yishun Ring Rd,"3-Room (Income Ceiling $7,000)",617,2022-05_SBF_617_02_3248,3248,02,73,Keys Available,-,213000,NaN,NaN,You may view floor plans and photographs (if a...


In [48]:
def process_ethnic_data(ethnic_quota):
    processed_datas = []
    for key, value in ethnic_quota.items():
        if key == "projectBlockFlatTypeInfoMap":     
            block = ethnic_quota["projectBlockFlatTypeInfoMap"]
            for blk1, flat_type1 in block.items():
                processed_data = {}
                processed_data["project_id"] =  ethnic_quota["project_id"]
                for flat_type2, blk2 in flat_type1.items():
                    processed_data["block"] = blk2["block"]
                    processed_data["chinese"] = blk2["chinese"]
                    processed_data["malay"] = blk2["malay"]
                    processed_data["indian"] = blk2["indian"]
                processed_datas.append(processed_data)
    return processed_datas
l = list(map(process_ethnic_data, ethnic_quota_data))
import itertools
flattened_l = list(itertools.chain(*l))

In [56]:
ethnic_quota_df = pd.DataFrame(flattened_l).rename(columns={"block": "block-number"})
ethnic_quota_df

,project_id,block-number,chinese,malay,indian
0,2022-05_SBF_Q0NLX040QzE0XzE2NTAzNDk4MDAwNTM,476A,2,2,2
1,2022-05_SBF_Q0NLX040QzE1XzE2NTMzNzM4MDAwNDA,488C,2,2,1
2,2022-05_SBF_Q0NLX041QzEyXzE2NTAzNDYyMDAwODg,573,2,2,2
3,2022-05_SBF_Q0NLX041QzEyXzE2NTAzNDYyMDAwODg,570,1,1,1
4,2022-05_SBF_Q0NLX041QzEyXzE2NTAzNDYyMDAwODg,571,1,1,1
...,...,...,...,...,...
778,2022-05_SBF_WVNfTjVDMl8xNjUwMzQ2MjAwMDgy,505A,1,1,1
779,2022-05_SBF_WVNfTjVDNF8xNjUwMzUzNDAwMDM1,509B,1,1,1
780,2022-05_SBF_WVNfTjVDNl8xNjUwMzQyNjAwMDI3,511A,1,1,1
781,2022-05_SBF_WVNfTjZDMl8xNjUwMzM5MDAwMTYy,617,1,1,1


In [92]:
flat_data_df_final = flat_data_df.merge(ethnic_quota_df, how="left", on=["project_id", "block-number"])
flat_data_df_final = flat_data_df_final.drop(columns=["price-tooltip", "re-purchase-tooltip"])

flat_data_df_final["level"] = pd.to_numeric(flat_data_df_final["level"])
flat_data_df_final["keys_available"] = flat_data_df_final["deliveryPossessionDate"] == "Keys Available"
flat_data_df_final["estimatedCompletionDate"] = pd.to_datetime(flat_data_df_final["estimatedCompletionDate"], errors="coerce")
flat_data_df_final["deliveryPossessionDate"] = pd.to_datetime(flat_data_df_final["deliveryPossessionDate"], errors="coerce")
flat_data_df_final.to_csv("may_sbf_2022_final.csv", index=False)

In [93]:
flat_data_df_final

,coordinates,project,region,town,project_id,remaining_lease,street_name,project_name,flat-type,block-number,...,level,area,deliveryPossessionDate,estimatedCompletionDate,price,short-lease-price,chinese,malay,indian,keys_available
0,"[1.3766745369726854, 103.73789561376547]",Sunshine Court,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE0XzE2NTAzNDk4MDAwNTM,15 - 45 years,Choa Chu Kang Ave 5,Sunshine Court,2-Room Flexi,476A,...,2,36,NaT,NaT,57200,NaN,2,2,2,True
1,"[1.3766745369726854, 103.73789561376547]",Sunshine Court,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE0XzE2NTAzNDk4MDAwNTM,15 - 45 years,Choa Chu Kang Ave 5,Sunshine Court,2-Room Flexi,476A,...,9,47,NaT,NaT,84400,NaN,2,2,2,True
2,"[1.3765873889910192, 103.74007753792313]",Sunshine Gardens,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE1XzE2NTMzNzM4MDAwNDA,94 years,Choa Chu Kang Ave 5,Sunshine Gardens,5-Room,488C,...,2,113,NaT,NaT,405000,NaN,2,2,1,True
3,"[1.3765873889910192, 103.74007753792313]",Sunshine Gardens,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX040QzE1XzE2NTMzNzM4MDAwNDA,94 years,Choa Chu Kang Ave 5,Sunshine Gardens,5-Room,488C,...,2,113,NaT,NaT,405000,NaN,2,2,1,True
4,"[1.3942738802398278, 103.74452794822368]",Limbang Green,WEST REGION,Choa Chu Kang,2022-05_SBF_Q0NLX041QzEyXzE2NTAzNDYyMDAwODg,15 - 45 years,Choa Chu Kang St 52,Limbang Green,2-Room Flexi,570,...,2,47,NaT,NaT,88500,NaN,1,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,"[1.418883880805233, 103.84487594211994]",Acacia Breeze @ Yishun,NORTH REGION,Yishun,2022-05_SBF_WVNfTjVDMl8xNjUwMzQ2MjAwMDgy,93 years,Yishun St 51,Acacia Breeze @ Yishun,"3-Room (Income Ceiling $7,000)",505A,...,7,68,NaT,NaT,253000,NaN,1,1,1,True
1948,"[1.413868249473606, 103.84013548258679]",Angsana Breeze @ Yishun,NORTH REGION,Yishun,2022-05_SBF_WVNfTjVDNF8xNjUwMzUzNDAwMDM1,95 years,Yishun Ave 4,Angsana Breeze @ Yishun,4-Room,509B,...,3,93,NaT,NaT,338000,NaN,1,1,1,True
1949,"[1.4156731558147704, 103.84143663027895]",Oleander Breeze @ Yishun,NORTH REGION,Yishun,2022-05_SBF_WVNfTjVDNl8xNjUwMzQyNjAwMDI3,95 years,Yishun St 51,Oleander Breeze @ Yishun,4-Room,511A,...,3,93,NaT,NaT,353000,NaN,1,1,1,True
1950,"[1.4193039750769822, 103.83535078902591]",Yishun Ring Rd,NORTH REGION,Yishun,2022-05_SBF_WVNfTjZDMl8xNjUwMzM5MDAwMTYy,65 years,Yishun Ring Rd,Yishun Ring Rd,"3-Room (Income Ceiling $7,000)",617,...,2,73,NaT,NaT,213000,NaN,1,1,1,True


# Scraping Application Rates

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
application_rate_url = "https://services2.hdb.gov.sg/webapp/BP13BTOENQWeb/AR_May2022_SBF?strSystem=SBF"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get(application_rate_url, headers=headers)
html = r.content
soup = BeautifulSoup(html)


In [7]:
application_rate_table = soup.find_all("tbody")[-1]

In [8]:
fields = ("project", "flat_type", "n_units", "n_applicants", "first_timers", "second_timers", "overall")
application_rate_table_data = application_rate_table.find_all("tr") 
application_rates = []
for row in application_rate_table_data:
    data = [d.text for d in row.find_all("td")]
    if len(data) == 7:
        project = data.pop(0)
    if len(data) < 6:
        continue
    flat_type_application_rate = {"project": project}
    for key, value in zip(fields[1:], data):
        flat_type_application_rate[key] = value
    application_rates.append(flat_type_application_rate)
del application_rates[-1]

In [9]:
# creating application rates df

application_rates_df = pd.DataFrame(application_rates)
application_rates_df.loc[application_rates_df["flat_type"].str.contains("3-room"), "flat_type"] = "3-room"
# application_rates_df[application_rates_df["flat_type"].str.contains("3-room")]
application_rates_df["flat_type"] = application_rates_df["flat_type"].str.split(" / ")
application_rates_df = application_rates_df.explode("flat_type")
application_rates_df["weight*data"] = pd.to_numeric(application_rates_df["n_units"]) * pd.to_numeric(application_rates_df["overall"])
sums_df = application_rates_df.groupby(["project", "flat_type"]).sum().reset_index()
application_rates_df = application_rates_df.merge(sums_df, on=["project", "flat_type"], how="left")
application_rates_df["application_rate"] = application_rates_df["weight*data_x"] / application_rates_df["weight*data_y"] * pd.to_numeric(application_rates_df["first_timers"])
application_rates_df = application_rates_df.groupby(["project", "flat_type"]).sum().reset_index()
application_rates_df = application_rates_df.drop(columns=["weight*data_x", "weight*data_y"])
application_rates_df = application_rates_df.rename(columns={"project": "town"})
application_rates_df.loc[application_rates_df["town"].str.contains("Jurong"), "town"] = application_rates_df[application_rates_df["town"].str.contains("Jurong")]["town"].str.split(" / ")
application_rates_df = application_rates_df.explode("town")
application_rates_df.loc[application_rates_df["town"] == "West", "town"] = "Jurong West"
application_rates_df["town"] = application_rates_df["town"].str.replace(" / ", "/")

In [11]:
application_rates_df.to_csv("application_rates.csv")